In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install vecstack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score, r2_score
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

from vecstack import StackingTransformer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from vecstack import stacking
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import accuracy_score




In [ ]:
# Load data
train_df = pd.read_csv('/gdrive/My Drive/Team_Assignment_1/train.csv')

test_df = pd.read_csv('/gdrive/My Drive/Team_Assignment_1/test.csv')
test_df_original = test_df.copy()

train_columns = list(train_df.columns)


print("Restaurant Revenue - Train Data Shape", train_df.shape)
print("Restaurant Revenue - Test Data Shape", test_df.shape)

print("No. of features in train set", len(train_columns))
test_columns = list(test_df.columns)

print("No. of features in test set", len(test_columns))
train_df.head()


Restaurant Revenue - Train Data Shape (137, 43)
Restaurant Revenue - Test Data Shape (100000, 42)
No. of features in train set 43
No. of features in test set 42


,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [ ]:
# Training Data Info
train_df.info()

#Test Data Info
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id          137 non-null    int64  
 1   Open Date   137 non-null    object 
 2   City        137 non-null    object 
 3   City Group  137 non-null    object 
 4   Type        137 non-null    object 
 5   P1          137 non-null    int64  
 6   P2          137 non-null    float64
 7   P3          137 non-null    float64
 8   P4          137 non-null    float64
 9   P5          137 non-null    int64  
 10  P6          137 non-null    int64  
 11  P7          137 non-null    int64  
 12  P8          137 non-null    int64  
 13  P9          137 non-null    int64  
 14  P10         137 non-null    int64  
 15  P11         137 non-null    int64  
 16  P12         137 non-null    int64  
 17  P13         137 non-null    float64
 18  P14         137 non-null    int64  
 19  P15         137 non-null    i

In [ ]:
#To get list of names of all Columns

TrainCols = list(train_df.columns.values)
TestCols = list(test_df.columns.values)
print(TrainCols)
print(TestCols)

['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37', 'revenue']
['Id', 'Open Date', 'City', 'City Group', 'Type', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25', 'P26', 'P27', 'P28', 'P29', 'P30', 'P31', 'P32', 'P33', 'P34', 'P35', 'P36', 'P37']


In [ ]:
# To check number of null values in train data
train_df.isna().sum()

Id            0
Open Date     0
City          0
City Group    0
Type          0
P1            0
P2            0
P3            0
P4            0
P5            0
P6            0
P7            0
P8            0
P9            0
P10           0
P11           0
P12           0
P13           0
P14           0
P15           0
P16           0
P17           0
P18           0
P19           0
P20           0
P21           0
P22           0
P23           0
P24           0
P25           0
P26           0
P27           0
P28           0
P29           0
P30           0
P31           0
P32           0
P33           0
P34           0
P35           0
P36           0
P37           0
revenue       0
dtype: int64

In [ ]:
# Converting the Open Date Column to the datetime data type
train_df['Open Date'] = pd.to_datetime(train_df['Open Date'])

# Add seperate columns for the DateTime data types

train_df['Opening Day'] = train_df['Open Date'].dt.day
train_df['Opening Year'] = train_df['Open Date'].dt.year
train_df['Opening Month'] = train_df['Open Date'].dt.month


test_df['Open Date'] = pd.to_datetime(test_df['Open Date'])

# Add seperate columns for the DateTime data types

test_df['Opening Day'] = test_df['Open Date'].dt.day
test_df['Opening Year'] = test_df['Open Date'].dt.year
test_df['Opening Month'] = test_df['Open Date'].dt.month

In [ ]:
#Dropping ID and Revenue Column from Train Data Set
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

Y = train_df['revenue']
X = train_df.drop(['revenue'], axis=1)

X_Test = test_df;
print(X.shape, Y.shape)

(137, 44) (137,)


In [ ]:
#List of Categorical Features
categoricalFeatures = ['City', 'City Group', 'Type', 'Opening Day', 'Opening Month', 'Opening Year']

In [ ]:
# OneHotEncoding on Train set (fit & transform)
# OneHotEncoding is to be done on Categorical variables.

ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(X[categoricalFeatures]),columns=ohe.get_feature_names(),index=X.index)
Xtrain = pd.concat([X,Xcat],axis=1) 
Xtrain.drop(labels=categoricalFeatures,axis=1,inplace=True)
Xtrain = Xtrain.drop(['Open Date'], axis = 1)
Xtrain.sample(5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,x5_2005,x5_2006,x5_2007,x5_2008,x5_2009,x5_2010,x5_2011,x5_2012,x5_2013,x5_2014
94,2,3.0,4.0,4.0,2,2,5,5,5,5,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
104,4,5.0,4.0,3.0,1,3,5,5,5,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,3,5.0,4.0,4.0,2,5,5,5,5,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,4,5.0,5.0,4.0,1,5,5,5,4,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,2,2.0,4.0,4.0,3,1,4,5,5,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# OneHotEncoding on Test Data set (fit & transform)
# OneHotEncoding is to be done on Categorical variables.

Xcat = pd.DataFrame(ohe.transform(X_Test[categoricalFeatures]),columns=ohe.get_feature_names(),index=X_Test.index)
XTest = pd.concat([X_Test,Xcat],axis=1) 
XTest.drop(labels=categoricalFeatures,axis=1,inplace=True)
XTest = XTest.drop(['Open Date'], axis = 1)
XTest.sample(5)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,x5_2005,x5_2006,x5_2007,x5_2008,x5_2009,x5_2010,x5_2011,x5_2012,x5_2013,x5_2014
84505,3,4.0,4.0,3.0,1,3,5,4,5,4,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
92175,2,3.0,4.0,4.0,2,2,2,4,4,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
35489,3,5.0,3.0,4.0,4,3,5,5,5,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97757,3,7.5,6.0,3.0,1,4,5,5,5,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
31317,6,4.0,4.0,4.0,1,6,10,4,5,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# Model #1: Random Forest Regressor

regr1 = RandomForestRegressor()
regr1 = RandomForestRegressor(n_estimators= 100, max_depth=25,max_leaf_nodes = 75)
regr1.fit(Xtrain, Y)

#Predicting the Revenue on the Test Data Set
rfr_predict_Test=regr1.predict(XTest)
print(rfr_predict_Test)

[3653259.93377381 4009436.08396825 4228506.06161111 ... 4311483.49266667
 4532886.51016667 4602807.265     ]


In [ ]:
#Hyperparameter Tuning for Random Forest Regressor

#Random Search

import time
start_time = time.time()

print("RandomizedSearchCV - Random Forest Regressor")
rand_parameters={'min_samples_leaf' : range(10,200,20),'max_depth': 
            range(1,50,6), 'max_leaf_nodes': 
            range(1,75,8)}

rf_random = RandomizedSearchCV(regr1,rand_parameters,n_iter=25,cv=5)
rf_random.fit(Xtrain, Y)
rand_parm=rf_random.best_params_
print(rand_parm)


#Grid Search

print("GridSearchCV - Random Forest Regressor")
rf_grid = GridSearchCV(regr1,rand_parameters)
rf_grid.fit(Xtrain, Y)
grid_parm1=rf_grid.best_params_
print(grid_parm1)

##Using the parameters obtained from HyperParameterTuning in the RandomForestRegressor

rfRand = RandomForestRegressor(**rand_parm)
rfGrid = RandomForestRegressor(**grid_parm1)

rfRand.fit(Xtrain,Y)
rfRand_predict = rfRand.predict(XTest)

rfGrid.fit(Xtrain,Y)
rfGrid_predict = rfGrid.predict(XTest)

#Writing the Predictions to CSV file
testData_id = test_df_original.Id


rfr_random_prediction = pd.DataFrame({"Id":testData_id, "Prediction":rfRand_predict})  
rfr_random_prediction.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/Random_Search_Submission.csv',index=None)


rfr_grid_prediction = pd.DataFrame({"Id":testData_id, "Prediction":rfGrid_predict})  
rfr_grid_prediction.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/Grid_Search_Submission.csv',index=None)

RandomizedSearchCV - Random Forest Regressor


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 125.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/par

{'min_samples_leaf': 10, 'max_leaf_nodes': 33, 'max_depth': 25}
GridSearchCV - Random Forest Regressor


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
450 fits failed out of a total of 4500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/

{'max_depth': 25, 'max_leaf_nodes': 49, 'min_samples_leaf': 10}


In [ ]:
#Model 2 : SVR Regressor

from sklearn.svm import SVR

regr2 =SVR(gamma=0.1,C=100.0, epsilon=0.1)
regr2.fit(Xtrain, Y)

#Predicting the Revenue on the Test Data Set
regr2_predict_Test=regr2.predict(XTest)
print(regr2.predict(XTest))

[3939770.11882467 3939750.56344247 3939796.10465338 ... 3939773.78836791
 3939772.90944653 3939772.90846693]


In [ ]:
#Hyper parameter tuning for SVR model

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],'kernel': ['rbf', 'linear', 'poly']} 
  
regr2_grid = GridSearchCV(regr2, param_grid, refit = True, verbose = 3)
regr2_random= RandomizedSearchCV(regr2,param_grid,n_iter=15)  

# Fitting the model for Grid Search
regr2_grid.fit(Xtrain, Y)
grid_parm=regr2_grid.best_params_
print(grid_parm)

#Fitting the model for Random Search
regr2_random.fit(Xtrain,Y)
random_parm=regr2_random.best_params_
print(random_parm)

#Using the best parameters obtained from HyperParameterTuning in the SVRClassifier 

SVRGrid = SVR(**grid_parm)
SVRGrid.fit(Xtrain,Y)
SVR_gridpredict = SVRGrid.predict(XTest)

SVRRand=SVR(**random_parm)
SVRRand.fit(Xtrain,Y)
SVR_randpredict = SVRRand.predict(XTest)

#Writing the Predictions to CSV file
testData_id = test_df_original.Id


SVR_prediction_grid = pd.DataFrame({"Id":testData_id, "Prediction":SVR_gridpredict})  
SVR_prediction_grid.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/SVR_Grid.csv',index=None)

SVR_prediction_random = pd.DataFrame({"Id":testData_id, "Prediction":SVR_randpredict})  
SVR_prediction_random.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/SVR_Random.csv',index=None)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.094 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.049 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.001 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.082 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.049 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.094 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.049 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.001 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.082 total time=   0.0s
[CV 5/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.049 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.094 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

In [ ]:
# Model 3: Decision Tree Regressor
regr3 = DecisionTreeRegressor(random_state = 42)
regr3.fit(Xtrain, Y)

#Predicting the Revenue on the Test Data Set
dt_predict_Test=regr3.predict(XTest)
print(dt_predict_Test)

# Hyper Parameter Tuning for Decision Tree Regressor
parameters = {
    'criterion': (['squared_error']),
    'splitter': ('best', 'random'),
    'max_depth': (8,16,32,64,128,256,512,1024,2048),
    'max_features': ('sqrt', 'log2'),
    'min_samples_leaf': ([1,2,3,4])
}

#Fitting the model on Random Search
test1 = RandomizedSearchCV(regr3, parameters, cv=5)
test1.fit(Xtrain, Y)

rand_params = test1.best_params_
print("The best parameters based on RandomizedSearchCV is", rand_params)
print()

#Fitting the model on Grid Search
test2 = GridSearchCV(regr3, parameters, cv=5)
test2.fit(Xtrain, Y)
print('The cross-validated training r-sq is: %.2f'% test2.score(Xtrain, Y))

grid_params = test2.best_params_
print("The best parameters based on GridSearchCV is", grid_params)

## Using the parameters obtained from HyperParameterTuning in the RandomForestRegressor
dtRand = DecisionTreeRegressor(**rand_params)
dtGrid = DecisionTreeRegressor(**grid_params)

dtRand.fit(Xtrain, Y)
dtRand_predict = dtRand.predict(XTest)
dtGrid.fit(Xtrain, Y)
dtGrid_predict = dtGrid.predict(XTest)

#Writing the Predictions to CSV file
testData_id = test_df_original.Id


dtr_prediction_random = pd.DataFrame({"Id":testData_id, "Prediction": dtRand_predict})
dtr_prediction_grid = pd.DataFrame({"Id":testData_id, "Prediction": dtGrid_predict})

dtr_prediction_random.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/dt_submission_random.csv',index=None)
dtr_prediction_grid.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/dt_submission_grid.csv',index=None)



[4067566. 4067566. 2025297. ... 3727364. 4350573. 3752885.]
The best parameters based on RandomizedSearchCV is {'splitter': 'best', 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 1024, 'criterion': 'squared_error'}

The cross-validated training r-sq is: 0.28
The best parameters based on GridSearchCV is {'criterion': 'squared_error', 'max_depth': 8, 'max_features': 'log2', 'min_samples_leaf': 3, 'splitter': 'best'}


In [ ]:
 # Model 4: MLP Regressor 
regr4 = MLPRegressor()
regr4 = MLPRegressor(random_state=1, max_iter=2000)
regr4.fit(Xtrain, Y)

#Predicting the test data
mlpr_predict_Test = regr4.predict(XTest)
print(mlpr_predict_Test)

print("MLP Randomized Search")

#hyperparameter tuning
rand_parameters = {'learning_rate' : ['adaptive', 'constant', 'invscaling'], 'activation' : ['logistic', 'relu', 'identity'], 'solver': ['lbfgs', 'sgd']}

#Fitting the model on Random Search
mlpr_random  = RandomizedSearchCV(regr4, rand_parameters, n_iter=25, cv=5)
mlpr_random.fit(Xtrain,Y)
rand_parm_mlp = mlpr_random.best_params_
print(rand_parm_mlp)

#Fitting the model on Grid Search
mlpr_grid = GridSearchCV(regr4, rand_parameters, cv=5)
mlpr_grid.fit(Xtrain,Y)
grid_parm_mlp = mlpr_grid.best_params_
print(grid_parm_mlp)

mlprRand = MLPRegressor(**rand_parm_mlp)
mlprGrid = MLPRegressor(**grid_parm_mlp)

#New Regressor with the best parameters
mlprRand.fit(Xtrain,Y)
mlprRand_predict = mlprRand.predict(XTest)

mlprGrid.fit(Xtrain,Y)
mlprGrid_predict = mlprGrid.predict(XTest)

testData_id = test_df_original.Id

mlpr_prediction_random = pd.DataFrame({"Id":testData_id, "Prediction":mlprRand_predict})
mlpr_prediction_grid = pd.DataFrame({"Id":testData_id, "Prediction":mlprGrid_predict})

mlpr_prediction_random.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/MLP_Random.csv',index=None)
mlpr_prediction_grid.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/MLP_Grid.csv',index=None)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


[40176.64549683 37580.1988955  45301.23965446 ... 73521.04966689
 91926.20446729 53505.5874903 ]
MLP Randomized Search


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 18 is smaller than n_iter=25. Running 18 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_mu

{'solver': 'sgd', 'learning_rate': 'invscaling', 'activation': 'logistic'}


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_m

{'activation': 'logistic', 'learning_rate': 'invscaling', 'solver': 'sgd'}


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
#Model 5: SGD Regressor

from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# Always scale the input. The most convenient way is to use a pipeline.
reg5 = make_pipeline(StandardScaler(), SGDRegressor(max_iter=2000, tol=1e-3))
reg5.fit(Xtrain, Y)
sgd_test_prediction = reg5.predict(XTest)
print(reg5.predict(XTest))

reg5_SGD = SGDRegressor()
reg5_SGD.fit(Xtrain, Y)
sgd_test_prediction_normal = reg5_SGD.predict(XTest)
print(reg5_SGD.predict(XTest))

[3781455.16013271 6877238.18133739 4393586.57760239 ... 8440802.87630603
 4629247.66628354 3077325.76752639]
[ 1.20241103e+11  1.37047690e+11  5.14504185e+10 ... -1.78687487e+11
 -3.43138992e+11  5.32733948e+10]


In [ ]:
#Hyperparameter tuning done for SGD Regressor
param_grid={'loss' : ['squared_error', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'], 
            'penalty' : ['l1', 'l2', 'elasticnet'], 
            'alpha' : [0.01, 0.1, 1, 10]}

#Grid Search
sgd_grid = GridSearchCV(reg5_SGD, param_grid, refit = True, verbose = 3)

#Random Search
sgd_random = RandomizedSearchCV(reg5_SGD, param_grid,n_iter=10,cv=5)

# Fitting the model for grid search
sgd_grid.fit(Xtrain, Y)
grid_parm_sgd=sgd_grid.best_params_
print(grid_parm_sgd)

#Fitting the model for random search
sgd_random.fit(Xtrain, Y)
random_parm_sgd=sgd_random.best_params_
print(random_parm_sgd)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END alpha=0.01, loss=squared_error, penalty=l1;, score=-11057858511.030 total time=   0.0s
[CV 2/5] END alpha=0.01, loss=squared_error, penalty=l1;, score=-6584117342.652 total time=   0.0s
[CV 3/5] END alpha=0.01, loss=squared_error, penalty=l1;, score=-14854597672.426 total time=   0.0s
[CV 4/5] END alpha=0.01, loss=squared_error, penalty=l1;, score=-31185825233.469 total time=   0.0s
[CV 5/5] END alpha=0.01, loss=squared_error, penalty=l1;, score=-5860129622.298 total time=   0.0s
[CV 1/5] END alpha=0.01, loss=squared_error, penalty=l2;, score=-16800463122.251 total time=   0.0s
[CV 2/5] END alpha=0.01, loss=squared_error, penalty=l2;, score=-2806227293.935 total time=   0.0s
[CV 3/5] END alpha=0.01, loss=squared_error, penalty=l2;, score=-30052934577.047 total time=   0.0s
[CV 4/5] END alpha=0.01, loss=squared_error, penalty=l2;, score=-63191925294.810 total time=   0.0s
[CV 5/5] END alpha=0.01, loss=squared_err

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[CV 1/5] END alpha=0.01, loss=huber, penalty=l1;, score=-2.061 total time=   0.2s
[CV 2/5] END alpha=0.01, loss=huber, penalty=l1;, score=-5.925 total time=   0.2s


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[CV 3/5] END alpha=0.01, loss=huber, penalty=l1;, score=-2.211 total time=   0.2s
[CV 4/5] END alpha=0.01, loss=huber, penalty=l1;, score=-3.263 total time=   0.1s


...

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[CV 2/5] END alpha=10, loss=epsilon_insensitive, penalty=elasticnet;, score=-5.933 total time=   0.0s
[CV 3/5] END alpha=10, loss=epsilon_insensitive, penalty=elasticnet;, score=-2.215 total time=   0.0s
[CV 4/5] END alpha=10, loss=epsilon_insensitive, penalty=elasticnet;, score=-3.267 total time=   0.0s
[CV 5/5] END alpha=10, loss=epsilon_insensitive, penalty=elasticnet;, score=-5.023 total time=   0.0s
[CV 1/5] END alpha=10, loss=squared_epsilon_insensitive, penalty=l1;, score=-2998157344.342 total time=   0.0s
[CV 2/5] END alpha=10, loss=squared_epsilon_insensitive, penalty=l1;, score=-25575453705.385 total time=   0.0s
[CV 3/5] END alpha=10, loss=squared_epsilon_insensitive, penalty=l1;, score=-15003929540.704 total time=   0.0s
[CV 4/5] END alpha=10, loss=squared_epsilon_insensitive, penalty=l1;, score=-5340287470.690 total time=   0.0s
[CV 5/5] END alpha=10, loss=squared_epsilon_insensitive, penalty=l1;, score=-203168054784.186 total time=   0.0s
[CV 1/5] END alpha=10, loss=squar

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linea

{'penalty': 'l1', 'loss': 'epsilon_insensitive', 'alpha': 1}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


In [ ]:
#Constructing SGD Regressor with the Best Params

reg5_grid_search = make_pipeline(StandardScaler(), SGDRegressor(**grid_parm_sgd))
reg5_grid_search.fit(Xtrain, Y)
sgd_test_prediction_grid_search = reg5_grid_search.predict(XTest)
print(reg5_grid_search.predict(XTest))

reg5_grid_random = make_pipeline(StandardScaler(), SGDRegressor(**random_parm_sgd))
reg5_grid_random.fit(Xtrain, Y)
sgd_test_prediction_random_search = reg5_grid_random.predict(XTest)
print(reg5_grid_random.predict(XTest))


#Writing the Predictions to CSV file
testData_id = test_df_original.Id


sgd_predictions_random = pd.DataFrame({"Id":testData_id, "Prediction":sgd_test_prediction_random_search})  
sgd_predictions_random.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/SGD_Random_Submission.csv',index=None)

sgd_predictions_grid = pd.DataFrame({"Id":testData_id, "Prediction":sgd_test_prediction_grid_search})  
sgd_predictions_grid.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/SGD_Grid_Submission.csv',index=None)




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[94.93867126 94.93867126 94.93867126 ... 94.93867126 94.93867126
 94.93867126]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


[94.93867126 94.93867126 94.93867126 ... 94.93867126 94.93867126
 94.93867126]


In [ ]:
# ***ENSEMBLE METHODS STACKING***

#Model 1

print("\n Ensemble Methods Predictions using MLP, SVR, RandomForest\n")

models = [ regr1, regr2, regr4]
      
S_Train, S_Test = stacking(models,                   
                           Xtrain, Y, XTest,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=mean_squared_error, 
    
                           n_folds=5, 
                 
                           stratified=False,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)



 Ensemble Methods Predictions using MLP, SVR, RandomForest

task:         [regression]
metric:       [mean_squared_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestRegressor]
    fold  0:  [9998982028219.36132812]
    fold  1:  [5020467786928.10449219]
    fold  2:  [5058852800675.19433594]
    fold  3:  [11612874379737.11132812]
    fold  4:  [5362227120717.17871094]
    ----
    MEAN:     [7410680823255.39062500] + [2821280198492.14306641]
    FULL:     [7412126722296.53320312]

model  1:     [SVR]
    fold  0:  [11981207454607.70312500]
    fold  1:  [3078628644064.07666016]
    fold  2:  [3344291986247.65869141]
    fold  3:  [13277890486023.03320312]
    fold  4:  [2867612241023.52148438]
    ----
    MEAN:     [6909926162393.19824219] + [4690453137229.66992188]
    FULL:     [6918977138844.91601562]

model  2:     [MLPRegressor]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


    fold  0:  [34744017077757.28125000]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


    fold  1:  [17796103012675.27734375]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


    fold  2:  [21798036697128.04296875]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


    fold  3:  [38167451457610.40625000]
    fold  4:  [16935853814272.08398438]
    ----
    MEAN:     [25888292411888.61718750] + [8849423531019.09765625]
    FULL:     [25893865662010.19140625]



/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [ ]:
#Fitting MLP Regressor into Stacked Train Data Set

mlp_StackedModel = MLPRegressor()
mlp_StackedModel.fit(S_Train, Y)

#Predicting the Stacked Test Data Set
mlp_predict_stacked=mlp_StackedModel.predict(S_Test)
print(mlp_StackedModel.predict(S_Test))

#Hyper Parameter Tuning of the Stacked Model (MLP since it had the best score)

#Random Search
print("RandomizedSearchCV - MLP Regressor")
rand_parameters = {'learning_rate' : ['adaptive', 'constant', 'invscaling'], 'activation' : ['logistic', 'relu', 'identity'], 'solver': ['lbfgs', 'sgd']}

mlp_random_stacked = RandomizedSearchCV(mlp_StackedModel,rand_parameters,n_iter=3,cv=5)
mlp_random_stacked.fit(S_Train, Y)
random_parm_stacked=mlp_random_stacked.best_params_
print(random_parm_stacked)

#Grid Search
print("GridSearchCV - MLP Regressor")

mlp_grid_stacked = GridSearchCV(mlp_StackedModel,rand_parameters,cv=5)
mlp_grid_stacked.fit(S_Train, Y)
grid_parm_stacked=mlp_grid_stacked.best_params_
print(grid_parm_stacked)

#Using the best parameters to construct a MLP Regressor
mlprRand_stacked = MLPRegressor(**random_parm_stacked)
mlprGrid_stacked = MLPRegressor(**grid_parm_stacked)

mlprRand_stacked.fit(S_Train,Y)
mlprRand_stacked_predict = mlprRand_stacked.predict(S_Test)

mlprGrid_stacked.fit(S_Train,Y)
mlprGrid_stacked_predict = mlprGrid_stacked.predict(S_Test)

testData_id = test_df_original.Id

mlpr_prediction_random_stacked = pd.DataFrame({"Id":testData_id, "Prediction":mlprRand_stacked_predict})
mlpr_prediction_grid_stacked = pd.DataFrame({"Id":testData_id, "Prediction":mlprGrid_stacked_predict})

mlpr_prediction_random_stacked.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/MLP_Random_Stacked.csv',index=None)
mlpr_prediction_grid_stacked.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/MLP_Grid_Stacked.csv',index=None)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_base.py:174: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:153: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/usr/local/lib/python3.7/dist-packages/sklearn/utils/extmath.py:153: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


[4149491.21373777 4484330.25951089 4048224.02963532 ... 4570211.61478768
 4538740.8315592  4546283.90607817]
RandomizedSearchCV - MLP Regressor


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/base.py", line 706, in score
    return r2_score(y, y_pred, sample_weight=sample_weight)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py"

{'solver': 'lbfgs', 'learning_rate': 'constant', 'activation': 'logistic'}
GridSearchCV - MLP Regressor


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maxi

{'activation': 'relu', 'learning_rate': 'adaptive', 'solver': 'lbfgs'}


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
# ***ENSEMBLE METHODS STACKING***

#Model 2

print("\n Ensemble Methods Predictions using Decision Tree, SGD, RandomForest\n")

models1 = [ regr1, regr3, reg5]
      
S_Train1, S_Test1 = stacking(models1,                   
                           Xtrain, Y, XTest,   
                           regression=True, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=mean_squared_error, 
    
                           n_folds=7, 
                 
                           stratified=False,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


 Ensemble Methods Predictions using Decision Tree, SGD, RandomForest

task:         [regression]
metric:       [mean_squared_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [RandomForestRegressor]
    fold  0:  [4628060015085.91601562]
    fold  1:  [14255802207898.24609375]
    fold  2:  [2689356229658.98974609]
    fold  3:  [4150719906428.93652344]
    fold  4:  [9564181042721.33789062]
    fold  5:  [8611745580505.67968750]
    fold  6:  [3938852842976.31152344]
    ----
    MEAN:     [6834102546467.91699219] + [3841940137409.03125000]
    FULL:     [6822332679119.01562500]

model  1:     [DecisionTreeRegressor]
    fold  0:  [12000155137315.09960938]
    fold  1:  [20298349716049.60156250]
    fold  2:  [2368586252908.70019531]
    fold  3:  [8877199593409.69921875]
    fold  4:  [15728744996254.26367188]
    fold  5:  [11352401212300.63085938]
    fold  6:  [4122773305958.26318359]
    ----
    MEAN:     [10678315744885.18164062] + [5800545542270.80078125]
 

In [ ]:
#Fitting Random Forest into Stacked Train Data Set

rfr_StackedModel = RandomForestRegressor()
rfr_StackedModel.fit(S_Train, Y)

#Predicting the Stacked Test Data Set
rfr_predict_stacked=rfr_StackedModel.predict(S_Test)
print(rfr_StackedModel.predict(S_Test))

#Hyper Parameter Tuning of the Stacked Model (Random Forest since it had the best score)

#Random Search
print("RandomSearchCV - Random Forest")
rand_parameters_stacked={'min_samples_leaf' : range(10,200,20),'max_depth': 
            range(1,50,6), 'max_leaf_nodes': 
            range(1,75,8)}

rfr_random_stacked = RandomizedSearchCV(rfr_StackedModel,rand_parameters_stacked,n_iter=5,cv=5)
rfr_random_stacked.fit(S_Train, Y)
random_parm_rfr_stacked=rfr_random_stacked.best_params_
print(random_parm_rfr_stacked)

#Grid Search
print("GridSearchCV - Random Forest")

rfr_grid_stacked = GridSearchCV(rfr_StackedModel,rand_parameters_stacked,cv=5)
rfr_grid_stacked.fit(S_Train, Y)
rfr_grid_parm_stacked=rfr_grid_stacked.best_params_
print(rfr_grid_parm_stacked)

#Using the best parameters to construct a Random Forest Regressor
rfrRand_stacked = RandomForestRegressor(**random_parm_rfr_stacked)
rfrGrid_stacked = RandomForestRegressor(**rfr_grid_parm_stacked)

rfrRand_stacked.fit(S_Train,Y)
rfrRand_stacked_predict = rfrRand_stacked.predict(S_Test)

rfrGrid_stacked.fit(S_Train,Y)
rfrGrid_stacked_predict = rfrGrid_stacked.predict(S_Test)

testData_id = test_df_original.Id

rfr_prediction_random_stacked = pd.DataFrame({"Id":testData_id, "Prediction":rfrRand_stacked_predict})
rfr_prediction_grid_stacked = pd.DataFrame({"Id":testData_id, "Prediction":rfrGrid_stacked_predict})

mlpr_prediction_random_stacked.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/RFR_Random_Stacked.csv',index=None)
mlpr_prediction_grid_stacked.to_csv(r'/gdrive/My Drive/Team_Assignment_1/Submissions/RFR_Grid_Stacked.csv',index=None)

[4499807.71 4499808.61 6272150.63 ... 5693367.06 6159165.   4980974.21]
RandomSearchCV - Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/parall

{'min_samples_leaf': 110, 'max_leaf_nodes': 57, 'max_depth': 7}
GridSearchCV - Random Forest


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
450 fits failed out of a total of 4500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "/usr/local/lib/python3.7/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.7/dist-packages/joblib/

{'max_depth': 1, 'max_leaf_nodes': 73, 'min_samples_leaf': 10}
